In [ ]:
import numpy as np
import sys 
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from ArtrayAnalysis import ArtFit

#newest calib
#2024.01.15
CalibNewUnopt = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\2024.01.15\Artray653nmCalib.txt"
CalibNewOptimized = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\2024.01.15\Artray653nmCalibOptimized.txt"
JSONnew = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\JSON_ArtrayBG_2024.01.16.json"

artUnopt = ArtFit(JSONnew, CalibNewUnopt, 1)
artOpt = ArtFit(JSONnew,CalibNewOptimized, 1)

#october not great
CalibOctober = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\2023.10.30\Pump653Calib.txt"
JSONoct = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\JSON_ArtrayBG_2023.10.30.json"

artOct = ArtFit(JSONoct, CalibOctober, 1)

### Mirror consistency

In [ ]:
import numpy as np
import sys 
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from ArtrayAnalysis import ArtFit
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
#"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\14.07.2023\MirrorConsistencyTest\01.tif"
directory = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\14.07.2023\MirrorConsistencyTest"+"\\"[0]
fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3,2/3), dpi=288)
filestring = r"{0:02d}.tif"
for ind in range(10):
    map, size = ArtFit.loadTIFF(directory+filestring.format(ind+1))
    x = np.arange(0,size[0], 1)
    y = np.arange(0,size[1], 1)
    xAr, yAr = np.meshgrid(x,y)
    ax.contour(xAr, yAr, map, levels=3)
    
ax.set_xlim((0, size[0]))
ax.set_ylim((0, size[1]))
ax.axis('equal')
ax.set_xlabel('px / 1')
ax.set_ylabel('px / 1')
#plt.show()

#background
#second part
directory = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\ArtrayCalib\CameraData\Artray\14.07.2023\MirrorConsistencyTest"+"\\"[0]
fig2, ax2= plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3,2/3), dpi=288)
filestring = r"{0:02d}.txt"
for ind in range(4):
    map = ArtFit.convertByteToInt(ArtFit.loadByteArr(directory+filestring.format(ind+11)), 200, 300)
    x = np.arange(0,size[0], 1)
    y = np.arange(0,size[1], 1)
    xAr, yAr = np.meshgrid(x,y)
    ax.contour(xAr, yAr, map, levels=3)
    
ax2.set_xlim((0, size[0]))
ax2.set_ylim((0, size[1]))
ax2.axis('equal')
ax2.set_xlabel('px / 1')
ax2.set_ylabel('px / 1')
